# MAIN SCRIPT

In [2]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
import psycopg2

print('Libraries Loaded')

def read_bed_file(bed_file):
    bed_positions = set()
    with open(bed_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip header lines if present
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 3:
                chrom = fields[0]
                try:
                    start = int(fields[1])
                    end = int(fields[2])
                except ValueError:
                    continue  # Skip this line if start or end position is not an integer
                for pos in range(start, end + 1):
                    bed_positions.add((chrom, pos))
    return bed_positions

def normalize_chrom_name(chrom):
    return chrom.split('_')[0]

def filter_vcf_file(vcf_file, bed_positions):
    filtered_vcf_records = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Preserve header lines in the output
                filtered_vcf_records.append(line)
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 2:
                raw_chrom = fields[0]
                chrom = normalize_chrom_name(raw_chrom)
                try:
                    pos = int(fields[1])
                except ValueError:
                    continue  # Skip this line if 'POS' is not an integer
                if (chrom, pos) in bed_positions:
                    filtered_vcf_records.append(line)
    return filtered_vcf_records

def write_filtered_vcf(filtered_vcf_records, output_file):
    with open(output_file, 'w') as f:
        for record in filtered_vcf_records:
            f.write(record)

def main():
    bed_file = r'C:/Users/GenepoweRx_Madhu/Downloads/BED_files/srinivas_sir_covered.bed'
    vcf_file = r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHCDPRGPTTL125_final.vcf'
    output_file = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHCDPRGPTTL125_final.vcf'

    bed_positions = read_bed_file(bed_file)
    filtered_vcf_records = filter_vcf_file(vcf_file, bed_positions)
    write_filtered_vcf(filtered_vcf_records, output_file)

if __name__ == "__main__":
    main()

print('Covered rows Extracted')    
    
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHCDPRGPTTL125_final.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR']]

vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf['GT'] = vcf['GT'].astype(str)

print('Zygosity Extracted')

vcf["Gene Name"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
vcf['Gene Name'] = vcf['Gene Name'].apply(lambda x: ','.join(set([segment.split(':')[0] for segment in x.split('|')])) if pd.notnull(x) else '')
vcf

vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['csq'] = vcf['CSQ'].str.split(',')
vcf = vcf.explode('csq')
vcf

########################################################### Required columns extraction from the CSQ column ####################
vcf['ClinVar_CLNDN'] = vcf['csq'].str.split('|').str[82]
vcf['CLIN_SIG'] = vcf['csq'].str.split('|').str[70]
vcf['ClinVar_CLNREVSTAT'] = vcf['csq'].str.split('|').str[81]
vcf['ClinVar'] = vcf['csq'].str.split('|').str[79]
vcf['HGVSC'] = vcf['csq'].str.split('|').str[10]
vcf['HGVSP'] = vcf['csq'].str.split('|').str[11]
vcf['PolyPhen'] = vcf['csq'].str.split('|').str[38]
vcf['BIOTYPE'] = vcf['csq'].str.split('|').str[7]
vcf['EXON'] = vcf['csq'].str.split('|').str[8]
vcf['INTRON'] = vcf['csq'].str.split('|').str[9]
vcf['Protein_position'] = vcf['csq'].str.split('|').str[14]
vcf['Amino_acids'] = vcf['csq'].str.split('|').str[15]
vcf['Codons'] = vcf['csq'].str.split('|').str[16]
vcf['STRAND'] = vcf['csq'].str.split('|').str[19]
vcf['PUBMED'] = vcf['csq'].str.split('|').str[73]
vcf['Consequence'] = vcf['csq'].str.split('|').str[1]
vcf['IMPACT'] = vcf['csq'].str.split('|').str[2]
vcf['SIFT'] = vcf['csq'].str.split('|').str[37]
################################################## Frequency columns extraction ################################################
vcf['AF'] = vcf['csq'].str.split('|').str[42]
vcf['AFR_AF'] = vcf['csq'].str.split('|').str[43]
vcf['AMR_AF'] = vcf['csq'].str.split('|').str[44]
vcf['EAS_AF'] = vcf['csq'].str.split('|').str[45]
vcf['EUR_AF'] = vcf['csq'].str.split('|').str[46]
vcf['SAS_AF'] = vcf['csq'].str.split('|').str[47]
vcf['gnomADe_AF'] = vcf['csq'].str.split('|').str[48]
vcf['gnomADe_AFR_AF'] = vcf['csq'].str.split('|').str[49]
vcf['gnomADe_AMR_AF'] = vcf['csq'].str.split('|').str[50]
vcf['gnomADe_ASJ_AF'] = vcf['csq'].str.split('|').str[51]
vcf['gnomADe_EAS_AF'] = vcf['csq'].str.split('|').str[52]
vcf['gnomADe_FIN_AF'] = vcf['csq'].str.split('|').str[53]
vcf['gnomADe_NFE_AF'] = vcf['csq'].str.split('|').str[54]
vcf['gnomADe_OTH_AF'] = vcf['csq'].str.split('|').str[55]
vcf['gnomADe_SAS_AF'] = vcf['csq'].str.split('|').str[56]
vcf['gnomADg_AF'] = vcf['csq'].str.split('|').str[57]
vcf['gnomADg_AFR_AF'] = vcf['csq'].str.split('|').str[58]
vcf['gnomADg_AMI_AF'] = vcf['csq'].str.split('|').str[59]
vcf['gnomADg_AMR_AF'] = vcf['csq'].str.split('|').str[60]
vcf['gnomADg_ASJ_AF'] = vcf['csq'].str.split('|').str[61]
vcf['gnomADg_EAS_AF'] = vcf['csq'].str.split('|').str[62]
vcf['gnomADg_FIN_AF'] = vcf['csq'].str.split('|').str[63]
vcf['gnomADg_MID_AF'] = vcf['csq'].str.split('|').str[64]
vcf['gnomADg_NFE_AF'] = vcf['csq'].str.split('|').str[65]
vcf['gnomADg_OTH_AF'] = vcf['csq'].str.split('|').str[66]
vcf['gnomADg_SAS_AF'] = vcf['csq'].str.split('|').str[67]
vcf['MAX_AF'] = vcf['csq'].str.split('|').str[68]
vcf['MAX_AF_POPS'] = vcf['csq'].str.split('|').str[69]

print('Required columns extracted')

vcf['Protein Position and Amino Acid'] = vcf['Amino_acids'].str[0] + vcf['Protein_position'] + np.where(vcf['Amino_acids'].str[-1] == vcf['Amino_acids'].str[0], '', vcf['Amino_acids'].str[-1])

vcf[['HGVSc', 'HGVSc (Transcript)']] = vcf['HGVSC'].str.split(':', 1, expand=True)
vcf[['HGVSp', 'HGVSp (Transcript)']] = vcf['HGVSP'].str.split(':', 1, expand=True)

vcf_final = vcf[['Gene Name', 'rsID','CHROM', 'POS', 'REF', 'ALT', 'Zygosity', 'Consequence', 'IMPACT',
          'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT',
          'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF',
       'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF',
       'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF',
       'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF',
       'gnomADe_SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF',
       'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF',
       'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF',
       'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE', 'EXON', 'INTRON',
          'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]


# Define the terms to remove
remove_terms = set(["not_specified", "not_provided"])

# Apply the filtering operation to 'Column1' only
vcf_final['ClinVar_CLNDN'] = vcf_final['ClinVar_CLNDN'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)

vcf_final['CLIN_SIG'] = vcf_final['CLIN_SIG'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)

vcf_final['ClinVar_CLNREVSTAT'] = vcf_final['ClinVar_CLNREVSTAT'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final = vcf_final.astype(str).applymap(lambda x: x.replace('&', ',').replace('_', ' '))

vcf_final['consequence'] = vcf_final['Consequence'].str.split(',').str[0]


df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')

merged_1 = pd.merge(vcf_final, df_1, on='consequence', how='left', sort=False)

df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/IMPACT.xlsx')

merged_2 = pd.merge(merged_1, df_2, on = 'IMPACT', how='left', sort=False)

merged_2 = merged_2[['Gene Name', 'rsID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]


df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/OneDrive/Desktop/cardiac_genes.xlsx')

merged_2['Gene_Match'] = 'No'
merged_2['Matched_Gene'] = ''
    
# Iterate through each gene in vcf['Gene']
for index, genes in merged_2['Gene Name'].iteritems():
    if isinstance(genes, str):
        gene_list = genes.split(',')
        for gene in gene_list:
            if gene in df_gene['Gene Name'].values:
                merged_2.at[index, 'Gene_Match'] = 'Yes'
                merged_2.at[index, 'Matched_Gene'] = gene
                break
    
    
merged_2 = merged_2[['Gene Name', 'Gene_Match', 'Matched_Gene', 'rsID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence', 'Consequence_score', 'IMPACT', 'IMPACT_score',
       'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc',
       'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)', 'GT', 'GQ', 'SDP',
       'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT',
       'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE', 'EXON', 'INTRON',
       'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]
merged_2['POS'] = merged_2['POS'].astype('int64')

print('Genes Mapped')

import pandas as pd
x = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/OneDrive/Desktop/Cardiac_Lit_final_hg38_hg37.xlsx')
x['chrom'] = x['Chrom-pos-Ref-Alt_38'].str.split(',')
x = x.explode('chrom')

x['CHROM'] = x['chrom'].str.split('-').str[0]

# Function to add 'chr' prefix conditionally
def add_chr_prefix(chrom):
    if pd.notnull(chrom) and chrom.strip() != '':
        return 'chr' + str(chrom)
    else:
        return chrom

# Applying the function to the 'chromosome' column
x['CHROM'] = x['CHROM'].apply(add_chr_prefix)
x['CHROM'] = x['CHROM'].str.strip()
x['CHROM'] = x['CHROM'].str.replace(r'\s+', '')
x['POS'] = x['chrom'].str.split('-').str[1]
x['REF'] = x['chrom'].str.split('-').str[2]
x['ALT'] = x['chrom'].str.split('-').str[3]


x.dropna(subset=['CHROM'], inplace=True)
# Drop rows with empty cells after removing leading and trailing whitespaces
x['CHROM'] = x['CHROM'].str.strip()
x['POS'] = x['POS'].str.strip()
# Dropping rows with empty cells and NaN values in both 'chromosome' and 'position' columns
x.dropna(subset=['CHROM', 'POS'], inplace=True)
df_3 = x[['CHROM', 'POS', 'REF', 'ALT']]
df_3['Literature'] = 'Yes'
df_3.drop_duplicates(subset='POS', inplace=True)
df_3['POS'] = df_3['POS'].astype('int64')
df_3 = df_3.reset_index()
df_3 = df_3[['CHROM', 'POS', 'REF', 'ALT', 'Literature']]

df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KAPA HyperExome_hg38_capture_targets (1).bed', sep = '\t', header = None)
df.columns = ['chromosome', 'Start_pos', 'End_pos', 'INFO']

df['Extended_Start_pos'] = df['Start_pos'] - 20
df['Extended_End_pos'] = df['End_pos'] + 20

df['gene_symbol'] = df['INFO'].str.extract(r'gene_symbol=([^;]+)')
df = df[['chromosome', 'Extended_Start_pos', 'Extended_End_pos', 'INFO', 'gene_symbol']]


# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end in ranges:
            if start <= pos <= end:
                return 'Covered'
    return 'Not_Covered'

# Step 3: Apply the function to create the new column in dataset2
df_3['Covered/Not_Covered'] = df_3.apply(check_coverage, axis=1)

df_3 = df_3[df_3['Covered/Not_Covered'] == 'Covered']

merged_2['POS'] = merged_2['POS'].astype('int64')
df_3['POS'] = df_3['POS'].astype('int64')
merged_3 = pd.merge(merged_2, df_3, on=['CHROM', 'POS', 'REF', 'ALT'], how='left', sort=False)
merged_3['Literature'] = merged_3['Literature'].fillna('No')

print('Literature variants Mapped')

merged_3 = merged_3[['Gene Name', 'Gene_Match', 'Matched_Gene', 'rsID', 'Literature', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]

# EXON & INTRON filtering
#merged_3.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL1_depth_vcf_processed.xlsx', index=False)

df = merged_3[merged_3['Gene_Match'] == 'Yes']

# Replace empty strings with '0/0' and convert numerical parts to integers
df['EXON'] = df['EXON'].replace('', '0/0')
df['EXON_Numerator'] = df['EXON'].apply(lambda x: x.split('/')[0] if '/' in x else 0)
df['EXON_Denominator'] = df['EXON'].apply(lambda x: x.split('/')[1] if '/' in x else 0)

df['INTRON'] = df['INTRON'].replace('', '0/0')
df['INTRON_Numerator'] = df['INTRON'].apply(lambda x: x.split('/')[0] if '/' in x else 0)
df['INTRON_Denominator'] = df['INTRON'].apply(lambda x: x.split('/')[1] if '/' in x else 0)

# Convert the data types of numerator and denominator columns to integers
df['EXON_Numerator'] = df['EXON_Numerator'].astype(int)
df['EXON_Denominator'] = df['EXON_Denominator'].astype(int)

df['INTRON_Numerator'] = df['INTRON_Numerator'].astype(int)
df['INTRON_Denominator'] = df['INTRON_Denominator'].astype(int)

# Initialize an empty DataFrame to store the final result
result_df = pd.DataFrame()

# Iterate over unique combinations of CHROM, POS, rsID, REF, ALT
for _, group_df in df.groupby(['CHROM', 'POS', 'REF', 'ALT']):
    # Check if EXON column has values
    if not group_df['EXON_Numerator'].eq(0).all():
        # Prioritize rows with values in EXON column
        result_df = pd.concat([result_df, group_df.sort_values(by=['EXON_Numerator'], ascending=False).head(1)])
    else:
        # If EXON is empty, prioritize rows with values in INTRON column
        if not group_df['INTRON_Numerator'].eq(0).all():
            result_df = pd.concat([result_df, group_df.sort_values(by=['INTRON_Numerator'], ascending=False).head(1)])
        else:
            # If both EXON and INTRON are empty, just concatenate the first row
            result_df = pd.concat([result_df, group_df.head(1)])
            
print('EXON and INTRON filter applied')

# Now result_df contains the rows you're looking for based on the specified logic
result_df = result_df.drop(['Gene_Match', 'Gene Name', 'EXON_Numerator', 'EXON_Denominator', 'INTRON_Numerator', 'INTRON_Denominator'], axis=1)

result_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL125_final.xlsx', index=False)

print('Dataframe Exported as excel')

Libraries Loaded
Covered rows Extracted
Zygosity Extracted
Required columns extracted
Genes Mapped
Literature variants Mapped
EXON and INTRON filter applied
Dataframe Exported as excel


# Changing the file name _final

In [ ]:
import os

def remove_final_suffix(folder_path):
    # List all files in the specified folder
    files = os.listdir(folder_path)

    for file_name in files:
        # Check if the file name contains "_final"
        if "_final" in file_name:
            # Create the new file name by replacing "_final" with an empty string
            new_file_name = file_name.replace("_final", "")

            # Create the full paths for the old and new file names
            old_path = os.path.join(folder_path, file_name)
            new_path = os.path.join(folder_path, new_file_name)

            # Rename the file
            os.rename(old_path, new_path)

            print(f'Renamed: {file_name} -> {new_file_name}')

# Replace 'your_folder_path' with the actual path of your folder
remove_final_suffix(r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/latest_outputs/')

In [3]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
import psycopg2

def read_bed_file(bed_file):
    bed_positions = set()
    with open(bed_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip header lines if present
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 3:
                chrom = fields[0]
                try:
                    start = int(fields[1])
                    end = int(fields[2])
                except ValueError:
                    continue  # Skip this line if start or end position is not an integer
                for pos in range(start, end + 1):
                    bed_positions.add((chrom, pos))
    return bed_positions

def normalize_chrom_name(chrom):
    return chrom.split('_')[0]

def filter_vcf_file(vcf_file, bed_positions):
    filtered_vcf_records = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Preserve header lines in the output
                filtered_vcf_records.append(line)
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 2:
                raw_chrom = fields[0]
                chrom = normalize_chrom_name(raw_chrom)
                try:
                    pos = int(fields[1])
                except ValueError:
                    continue  # Skip this line if 'POS' is not an integer
                if (chrom, pos) in bed_positions:
                    filtered_vcf_records.append(line)
    return filtered_vcf_records

def write_filtered_vcf(filtered_vcf_records, output_file):
    with open(output_file, 'w') as f:
        for record in filtered_vcf_records:
            f.write(record)

def main():
    bed_file = r'C:/Users/GenepoweRx_Madhu/Downloads/BED_files/kalyani_mam_covered.bed'
    vcf_file = r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/KHCDPRGPTTL38.vcf'
    output_file = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHCDPRGPTTL38.vcf'

    bed_positions = read_bed_file(bed_file)
    filtered_vcf_records = filter_vcf_file(vcf_file, bed_positions)
    write_filtered_vcf(filtered_vcf_records, output_file)

if __name__ == "__main__":
    main()

vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHCDPRGPTTL38.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR']]
vcf

vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf['GT'] = vcf['GT'].astype(str)
vcf

vcf["Gene Name"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
vcf['Gene Name'] = vcf['Gene Name'].apply(lambda x: ','.join(set([segment.split(':')[0] for segment in x.split('|')])) if pd.notnull(x) else '')
vcf

vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['csq'] = vcf['CSQ'].str.split(',')
vcf = vcf.explode('csq')
vcf

########################################################### Required columns extraction from the CSQ column ####################
vcf['ClinVar_CLNDN'] = vcf['csq'].str.split('|').str[82]
vcf['CLIN_SIG'] = vcf['csq'].str.split('|').str[70]
vcf['ClinVar_CLNREVSTAT'] = vcf['csq'].str.split('|').str[81]
vcf['ClinVar'] = vcf['csq'].str.split('|').str[79]
vcf['HGVSC'] = vcf['csq'].str.split('|').str[10]
vcf['HGVSP'] = vcf['csq'].str.split('|').str[11]
vcf['PolyPhen'] = vcf['csq'].str.split('|').str[38]
vcf['BIOTYPE'] = vcf['csq'].str.split('|').str[7]
vcf['EXON'] = vcf['csq'].str.split('|').str[8]
vcf['INTRON'] = vcf['csq'].str.split('|').str[9]
vcf['Protein_position'] = vcf['csq'].str.split('|').str[14]
vcf['Amino_acids'] = vcf['csq'].str.split('|').str[15]
vcf['Codons'] = vcf['csq'].str.split('|').str[16]
vcf['STRAND'] = vcf['csq'].str.split('|').str[19]
vcf['PUBMED'] = vcf['csq'].str.split('|').str[73]
vcf['Consequence'] = vcf['csq'].str.split('|').str[1]
vcf['IMPACT'] = vcf['csq'].str.split('|').str[2]
vcf['SIFT'] = vcf['csq'].str.split('|').str[37]
################################################## Frequency columns extraction ################################################
vcf['AF'] = vcf['csq'].str.split('|').str[42]
vcf['AFR_AF'] = vcf['csq'].str.split('|').str[43]
vcf['AMR_AF'] = vcf['csq'].str.split('|').str[44]
vcf['EAS_AF'] = vcf['csq'].str.split('|').str[45]
vcf['EUR_AF'] = vcf['csq'].str.split('|').str[46]
vcf['SAS_AF'] = vcf['csq'].str.split('|').str[47]
vcf['gnomADe_AF'] = vcf['csq'].str.split('|').str[48]
vcf['gnomADe_AFR_AF'] = vcf['csq'].str.split('|').str[49]
vcf['gnomADe_AMR_AF'] = vcf['csq'].str.split('|').str[50]
vcf['gnomADe_ASJ_AF'] = vcf['csq'].str.split('|').str[51]
vcf['gnomADe_EAS_AF'] = vcf['csq'].str.split('|').str[52]
vcf['gnomADe_FIN_AF'] = vcf['csq'].str.split('|').str[53]
vcf['gnomADe_NFE_AF'] = vcf['csq'].str.split('|').str[54]
vcf['gnomADe_OTH_AF'] = vcf['csq'].str.split('|').str[55]
vcf['gnomADe_SAS_AF'] = vcf['csq'].str.split('|').str[56]
vcf['gnomADg_AF'] = vcf['csq'].str.split('|').str[57]
vcf['gnomADg_AFR_AF'] = vcf['csq'].str.split('|').str[58]
vcf['gnomADg_AMI_AF'] = vcf['csq'].str.split('|').str[59]
vcf['gnomADg_AMR_AF'] = vcf['csq'].str.split('|').str[60]
vcf['gnomADg_ASJ_AF'] = vcf['csq'].str.split('|').str[61]
vcf['gnomADg_EAS_AF'] = vcf['csq'].str.split('|').str[62]
vcf['gnomADg_FIN_AF'] = vcf['csq'].str.split('|').str[63]
vcf['gnomADg_MID_AF'] = vcf['csq'].str.split('|').str[64]
vcf['gnomADg_NFE_AF'] = vcf['csq'].str.split('|').str[65]
vcf['gnomADg_OTH_AF'] = vcf['csq'].str.split('|').str[66]
vcf['gnomADg_SAS_AF'] = vcf['csq'].str.split('|').str[67]
vcf['MAX_AF'] = vcf['csq'].str.split('|').str[68]
vcf['MAX_AF_POPS'] = vcf['csq'].str.split('|').str[69]
vcf

vcf['Protein Position and Amino Acid'] = vcf['Amino_acids'].str[0] + vcf['Protein_position'] + np.where(vcf['Amino_acids'].str[-1] == vcf['Amino_acids'].str[0], '', vcf['Amino_acids'].str[-1])
vcf

vcf[['HGVSc', 'HGVSc (Transcript)']] = vcf['HGVSC'].str.split(':', 1, expand=True)
vcf[['HGVSp', 'HGVSp (Transcript)']] = vcf['HGVSP'].str.split(':', 1, expand=True)
vcf

vcf_final = vcf[['Gene Name', 'rsID','CHROM', 'POS', 'REF', 'ALT', 'Zygosity', 'Consequence', 'IMPACT',
          'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT',
          'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF',
       'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF',
       'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF',
       'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF',
       'gnomADe_SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF',
       'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF',
       'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF',
       'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE', 'EXON', 'INTRON',
          'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]
vcf_final

# Define the terms to remove
remove_terms = set(["not_specified", "not_provided"])

# Apply the filtering operation to 'Column1' only
vcf_final['ClinVar_CLNDN'] = vcf_final['ClinVar_CLNDN'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['CLIN_SIG'] = vcf_final['CLIN_SIG'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['ClinVar_CLNREVSTAT'] = vcf_final['ClinVar_CLNREVSTAT'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)

# Print the modified DataFrame
vcf_final

vcf_final = vcf_final.astype(str).applymap(lambda x: x.replace('&', ',').replace('_', ' '))
vcf_final

vcf_final['consequence'] = vcf_final['Consequence'].str.split(',').str[0]
vcf_final

df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')
df_1

merged_1 = pd.merge(vcf_final, df_1, on='consequence', how='left', sort=False)
merged_1

df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/IMPACT.xlsx')


merged_2 = pd.merge(merged_1, df_2, on = 'IMPACT', how='left', sort=False)


merged_2 = merged_2[['Gene Name', 'rsID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]

df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/cardiac_genes.xlsx')

merged_2['Gene_Match'] = 'No'
merged_2['Matched_Gene'] = ''
    
# Iterate through each gene in vcf['Gene']
for index, genes in merged_2['Gene Name'].iteritems():
    if isinstance(genes, str):
        gene_list = genes.split(',')
        for gene in gene_list:
            if gene in df_gene['Gene Name'].values:
                merged_2.at[index, 'Gene_Match'] = 'Yes'
                merged_2.at[index, 'Matched_Gene'] = gene
                break
    

merged_2 = merged_2[['Gene Name', 'Gene_Match', 'Matched_Gene', 'rsID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence', 'Consequence_score', 'IMPACT', 'IMPACT_score',
       'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc',
       'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)', 'GT', 'GQ', 'SDP',
       'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT',
       'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE', 'EXON', 'INTRON',
       'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]
merged_2['POS'] = merged_2['POS'].astype('int64')

import pandas as pd
x = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Cardiac_Lit_final_hg38_hg37.xlsx')
x['chrom'] = x['Chrom-pos-Ref-Alt_38'].str.split(',')
x = x.explode('chrom')

x['CHROM'] = x['chrom'].str.split('-').str[0]

# Function to add 'chr' prefix conditionally
def add_chr_prefix(chrom):
    if pd.notnull(chrom) and chrom.strip() != '':
        return 'chr' + str(chrom)
    else:
        return chrom

# Applying the function to the 'chromosome' column
x['CHROM'] = x['CHROM'].apply(add_chr_prefix)
x['CHROM'] = x['CHROM'].str.strip()
x['CHROM'] = x['CHROM'].str.replace(r'\s+', '')
x['POS'] = x['chrom'].str.split('-').str[1]
x['REF'] = x['chrom'].str.split('-').str[2]
x['ALT'] = x['chrom'].str.split('-').str[3]


x.dropna(subset=['CHROM'], inplace=True)
# Drop rows with empty cells after removing leading and trailing whitespaces
x['CHROM'] = x['CHROM'].str.strip()
x['POS'] = x['POS'].str.strip()
# Dropping rows with empty cells and NaN values in both 'chromosome' and 'position' columns
x.dropna(subset=['CHROM', 'POS'], inplace=True)
df_3 = x[['CHROM', 'POS', 'REF', 'ALT']]
df_3['Literature'] = 'Yes'
df_3.drop_duplicates(subset='POS', inplace=True)
df_3['POS'] = df_3['POS'].astype('int64')
df_3 = df_3.reset_index()
df_3 = df_3[['CHROM', 'POS', 'REF', 'ALT', 'Literature']]

df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KAPA HyperExome_hg38_capture_targets (1).bed', sep = '\t', header = None)
df.columns = ['chromosome', 'Start_pos', 'End_pos', 'INFO']

df['Extended_Start_pos'] = df['Start_pos'] - 20
df['Extended_End_pos'] = df['End_pos'] + 20

df['gene_symbol'] = df['INFO'].str.extract(r'gene_symbol=([^;]+)')
df = df[['chromosome', 'Extended_Start_pos', 'Extended_End_pos', 'INFO', 'gene_symbol']]


# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end in ranges:
            if start <= pos <= end:
                return 'Covered'
    return 'Not_Covered'

# Step 3: Apply the function to create the new column in dataset2
df_3['Covered/Not_Covered'] = df_3.apply(check_coverage, axis=1)

df_3 = df_3[df_3['Covered/Not_Covered'] == 'Covered']

merged_2['POS'] = merged_2['POS'].astype('int64')
df_3['POS'] = df_3['POS'].astype('int64')
merged_3 = pd.merge(merged_2, df_3, on=['CHROM', 'POS', 'REF', 'ALT'], how='left', sort=False)
merged_3['Literature'] = merged_3['Literature'].fillna('No')

merged_3 = merged_3[['Gene Name', 'Gene_Match', 'Matched_Gene', 'rsID', 'Literature', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]

# EXON & INTRON filtering
#merged_3.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHHSPTGPCSP26_depth_vcf_processed.xlsx', index=False)

df = merged_3[merged_3['Gene_Match'] == 'Yes']
df['EXON'] = df['EXON'].replace('', '0/0')
df['INTRON'] = df['INTRON'].replace('', '0/0')

# Replace empty strings with '0/0'
df['EXON'] = df['EXON'].replace('', '0/0')
df['INTRON'] = df['INTRON'].replace('', '0/0')

# Convert the data types of EXON and INTRON to integers
df['EXON'] = df['EXON'].astype(str).astype(int, errors='ignore')
df['INTRON'] = df['INTRON'].astype(str).astype(int, errors='ignore')

# Initialize an empty DataFrame to store the final result
result_df = pd.DataFrame()

# Iterate over unique combinations of CHROM, POS, rsID, REF, ALT
for _, group_df in df.groupby(['CHROM', 'POS', 'REF', 'ALT']):
    # Check if EXON column has values
    if not group_df['EXON'].isnull().all():
        # Prioritize rows with values in EXON column
        result_df = pd.concat([result_df, group_df.sort_values(by=['EXON'], ascending=False).head(1)])
    else:
        # If EXON is empty, prioritize rows with values in INTRON column
        if not group_df['INTRON'].isnull().all():
            result_df = pd.concat([result_df, group_df.sort_values(by=['INTRON'], ascending=True).head(1)])
        else:
            # If both EXON and INTRON are empty, just concatenate the first row
            result_df = pd.concat([result_df, group_df.head(1)])

# Now result_df contains the rows you're looking for based on the specified logic
result_df = result_df.drop(['Gene_Match', 'Gene Name'], axis=1)
result_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/KHCDPRGPTTL38.xlsx', index=False)

# main code

In [13]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
import psycopg2

def read_bed_file(bed_file):
    bed_positions = set()
    with open(bed_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip header lines if present
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 3:
                chrom = fields[0]
                try:
                    start = int(fields[1])
                    end = int(fields[2])
                except ValueError:
                    continue  # Skip this line if start or end position is not an integer
                for pos in range(start, end + 1):
                    bed_positions.add((chrom, pos))
    return bed_positions

def normalize_chrom_name(chrom):
    return chrom.split('_')[0]

def filter_vcf_file(vcf_file, bed_positions):
    filtered_vcf_records = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Preserve header lines in the output
                filtered_vcf_records.append(line)
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 2:
                raw_chrom = fields[0]
                chrom = normalize_chrom_name(raw_chrom)
                try:
                    pos = int(fields[1])
                except ValueError:
                    continue  # Skip this line if 'POS' is not an integer
                if (chrom, pos) in bed_positions:
                    filtered_vcf_records.append(line)
    return filtered_vcf_records

def write_filtered_vcf(filtered_vcf_records, output_file):
    with open(output_file, 'w') as f:
        for record in filtered_vcf_records:
            f.write(record)

def main():
    bed_file = r'C:/Users/GenepoweRx_Madhu/Downloads/BED_files/kalyani_mam_covered.bed'
    vcf_file = r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/KHCDPRGPTTL39.vcf'
    output_file = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHCDPRGPTTL39.vcf'

    bed_positions = read_bed_file(bed_file)
    filtered_vcf_records = filter_vcf_file(vcf_file, bed_positions)
    write_filtered_vcf(filtered_vcf_records, output_file)

if __name__ == "__main__":
    main()

vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHCDPRGPTTL39.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR']]
vcf

vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf['GT'] = vcf['GT'].astype(str)
vcf

vcf["Gene Name"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
vcf['Gene Name'] = vcf['Gene Name'].apply(lambda x: ','.join(set([segment.split(':')[0] for segment in x.split('|')])) if pd.notnull(x) else '')
vcf

vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['csq'] = vcf['CSQ'].str.split(',')
vcf = vcf.explode('csq')
vcf

########################################################### Required columns extraction from the CSQ column ####################
vcf['ClinVar_CLNDN'] = vcf['csq'].str.split('|').str[82]
vcf['CLIN_SIG'] = vcf['csq'].str.split('|').str[70]
vcf['ClinVar_CLNREVSTAT'] = vcf['csq'].str.split('|').str[81]
vcf['ClinVar'] = vcf['csq'].str.split('|').str[79]
vcf['HGVSC'] = vcf['csq'].str.split('|').str[10]
vcf['HGVSP'] = vcf['csq'].str.split('|').str[11]
vcf['PolyPhen'] = vcf['csq'].str.split('|').str[38]
vcf['BIOTYPE'] = vcf['csq'].str.split('|').str[7]
vcf['EXON'] = vcf['csq'].str.split('|').str[8]
vcf['INTRON'] = vcf['csq'].str.split('|').str[9]
vcf['Protein_position'] = vcf['csq'].str.split('|').str[14]
vcf['Amino_acids'] = vcf['csq'].str.split('|').str[15]
vcf['Codons'] = vcf['csq'].str.split('|').str[16]
vcf['STRAND'] = vcf['csq'].str.split('|').str[19]
vcf['PUBMED'] = vcf['csq'].str.split('|').str[73]
vcf['Consequence'] = vcf['csq'].str.split('|').str[1]
vcf['IMPACT'] = vcf['csq'].str.split('|').str[2]
vcf['SIFT'] = vcf['csq'].str.split('|').str[37]
################################################## Frequency columns extraction ################################################
vcf['AF'] = vcf['csq'].str.split('|').str[42]
vcf['AFR_AF'] = vcf['csq'].str.split('|').str[43]
vcf['AMR_AF'] = vcf['csq'].str.split('|').str[44]
vcf['EAS_AF'] = vcf['csq'].str.split('|').str[45]
vcf['EUR_AF'] = vcf['csq'].str.split('|').str[46]
vcf['SAS_AF'] = vcf['csq'].str.split('|').str[47]
vcf['gnomADe_AF'] = vcf['csq'].str.split('|').str[48]
vcf['gnomADe_AFR_AF'] = vcf['csq'].str.split('|').str[49]
vcf['gnomADe_AMR_AF'] = vcf['csq'].str.split('|').str[50]
vcf['gnomADe_ASJ_AF'] = vcf['csq'].str.split('|').str[51]
vcf['gnomADe_EAS_AF'] = vcf['csq'].str.split('|').str[52]
vcf['gnomADe_FIN_AF'] = vcf['csq'].str.split('|').str[53]
vcf['gnomADe_NFE_AF'] = vcf['csq'].str.split('|').str[54]
vcf['gnomADe_OTH_AF'] = vcf['csq'].str.split('|').str[55]
vcf['gnomADe_SAS_AF'] = vcf['csq'].str.split('|').str[56]
vcf['gnomADg_AF'] = vcf['csq'].str.split('|').str[57]
vcf['gnomADg_AFR_AF'] = vcf['csq'].str.split('|').str[58]
vcf['gnomADg_AMI_AF'] = vcf['csq'].str.split('|').str[59]
vcf['gnomADg_AMR_AF'] = vcf['csq'].str.split('|').str[60]
vcf['gnomADg_ASJ_AF'] = vcf['csq'].str.split('|').str[61]
vcf['gnomADg_EAS_AF'] = vcf['csq'].str.split('|').str[62]
vcf['gnomADg_FIN_AF'] = vcf['csq'].str.split('|').str[63]
vcf['gnomADg_MID_AF'] = vcf['csq'].str.split('|').str[64]
vcf['gnomADg_NFE_AF'] = vcf['csq'].str.split('|').str[65]
vcf['gnomADg_OTH_AF'] = vcf['csq'].str.split('|').str[66]
vcf['gnomADg_SAS_AF'] = vcf['csq'].str.split('|').str[67]
vcf['MAX_AF'] = vcf['csq'].str.split('|').str[68]
vcf['MAX_AF_POPS'] = vcf['csq'].str.split('|').str[69]
vcf

vcf['Protein Position and Amino Acid'] = vcf['Amino_acids'].str[0] + vcf['Protein_position'] + np.where(vcf['Amino_acids'].str[-1] == vcf['Amino_acids'].str[0], '', vcf['Amino_acids'].str[-1])
vcf

vcf[['HGVSc', 'HGVSc (Transcript)']] = vcf['HGVSC'].str.split(':', 1, expand=True)
vcf[['HGVSp', 'HGVSp (Transcript)']] = vcf['HGVSP'].str.split(':', 1, expand=True)
vcf

vcf_final = vcf[['Gene Name', 'rsID','CHROM', 'POS', 'REF', 'ALT', 'Zygosity', 'Consequence', 'IMPACT',
          'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT',
          'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF',
       'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF',
       'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF',
       'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF',
       'gnomADe_SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF',
       'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF',
       'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF',
       'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE', 'EXON', 'INTRON',
          'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]
vcf_final

# Define the terms to remove
remove_terms = set(["not_specified", "not_provided"])

# Apply the filtering operation to 'Column1' only
vcf_final['ClinVar_CLNDN'] = vcf_final['ClinVar_CLNDN'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['CLIN_SIG'] = vcf_final['CLIN_SIG'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['ClinVar_CLNREVSTAT'] = vcf_final['ClinVar_CLNREVSTAT'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)

# Print the modified DataFrame
vcf_final

vcf_final = vcf_final.astype(str).applymap(lambda x: x.replace('&', ',').replace('_', ' '))
vcf_final

vcf_final['consequence'] = vcf_final['Consequence'].str.split(',').str[0]
vcf_final

df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')
df_1

merged_1 = pd.merge(vcf_final, df_1, on='consequence', how='left', sort=False)
merged_1

df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/IMPACT.xlsx')


merged_2 = pd.merge(merged_1, df_2, on = 'IMPACT', how='left', sort=False)


merged_2 = merged_2[['Gene Name', 'rsID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]

df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/cardiac_genes.xlsx')

merged_2['Gene_Match'] = 'No'
merged_2['Matched_Gene'] = ''
    
# Iterate through each gene in vcf['Gene']
for index, genes in merged_2['Gene Name'].iteritems():
    if isinstance(genes, str):
        gene_list = genes.split(',')
        for gene in gene_list:
            if gene in df_gene['Gene Name'].values:
                merged_2.at[index, 'Gene_Match'] = 'Yes'
                merged_2.at[index, 'Matched_Gene'] = gene
                break
    

merged_2 = merged_2[['Gene Name', 'Gene_Match', 'Matched_Gene', 'rsID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence', 'Consequence_score', 'IMPACT', 'IMPACT_score',
       'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc',
       'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)', 'GT', 'GQ', 'SDP',
       'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT',
       'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE', 'EXON', 'INTRON',
       'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]
merged_2['POS'] = merged_2['POS'].astype('int64')

import pandas as pd
x = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Cardiac_Lit_final_hg38_hg37.xlsx')
x['chrom'] = x['Chrom-pos-Ref-Alt_38'].str.split(',')
x = x.explode('chrom')

x['CHROM'] = x['chrom'].str.split('-').str[0]

# Function to add 'chr' prefix conditionally
def add_chr_prefix(chrom):
    if pd.notnull(chrom) and chrom.strip() != '':
        return 'chr' + str(chrom)
    else:
        return chrom

# Applying the function to the 'chromosome' column
x['CHROM'] = x['CHROM'].apply(add_chr_prefix)
x['CHROM'] = x['CHROM'].str.strip()
x['CHROM'] = x['CHROM'].str.replace(r'\s+', '')
x['POS'] = x['chrom'].str.split('-').str[1]
x['REF'] = x['chrom'].str.split('-').str[2]
x['ALT'] = x['chrom'].str.split('-').str[3]


x.dropna(subset=['CHROM'], inplace=True)
# Drop rows with empty cells after removing leading and trailing whitespaces
x['CHROM'] = x['CHROM'].str.strip()
x['POS'] = x['POS'].str.strip()
# Dropping rows with empty cells and NaN values in both 'chromosome' and 'position' columns
x.dropna(subset=['CHROM', 'POS'], inplace=True)
df_3 = x[['CHROM', 'POS', 'REF', 'ALT']]
df_3['Literature'] = 'Yes'
df_3.drop_duplicates(subset='POS', inplace=True)
df_3['POS'] = df_3['POS'].astype('int64')
df_3 = df_3.reset_index()
df_3 = df_3[['CHROM', 'POS', 'REF', 'ALT', 'Literature']]

df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KAPA HyperExome_hg38_capture_targets (1).bed', sep = '\t', header = None)
df.columns = ['chromosome', 'Start_pos', 'End_pos', 'INFO']

df['Extended_Start_pos'] = df['Start_pos'] - 20
df['Extended_End_pos'] = df['End_pos'] + 20

df['gene_symbol'] = df['INFO'].str.extract(r'gene_symbol=([^;]+)')
df = df[['chromosome', 'Extended_Start_pos', 'Extended_End_pos', 'INFO', 'gene_symbol']]


# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end in ranges:
            if start <= pos <= end:
                return 'Covered'
    return 'Not_Covered'

# Step 3: Apply the function to create the new column in dataset2
df_3['Covered/Not_Covered'] = df_3.apply(check_coverage, axis=1)

df_3 = df_3[df_3['Covered/Not_Covered'] == 'Covered']

merged_2['POS'] = merged_2['POS'].astype('int64')
df_3['POS'] = df_3['POS'].astype('int64')
merged_3 = pd.merge(merged_2, df_3, on=['CHROM', 'POS', 'REF', 'ALT'], how='left', sort=False)
merged_3['Literature'] = merged_3['Literature'].fillna('No')

merged_3 = merged_3[['Gene Name', 'Gene_Match', 'Matched_Gene', 'rsID', 'Literature', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF',
       'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]

# EXON & INTRON filtering
#merged_3.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL1_depth_vcf_processed.xlsx', index=False)

df = merged_3[merged_3['Gene_Match'] == 'Yes']

# Replace empty strings with '0/0' and convert numerical parts to integers
df['EXON'] = df['EXON'].replace('', '0/0')
df['EXON_Numerator'] = df['EXON'].apply(lambda x: x.split('/')[0] if '/' in x else 0)
df['EXON_Denominator'] = df['EXON'].apply(lambda x: x.split('/')[1] if '/' in x else 0)

df['INTRON'] = df['INTRON'].replace('', '0/0')
df['INTRON_Numerator'] = df['INTRON'].apply(lambda x: x.split('/')[0] if '/' in x else 0)
df['INTRON_Denominator'] = df['INTRON'].apply(lambda x: x.split('/')[1] if '/' in x else 0)

# Convert the data types of numerator and denominator columns to integers
df['EXON_Numerator'] = df['EXON_Numerator'].astype(int)
df['EXON_Denominator'] = df['EXON_Denominator'].astype(int)

df['INTRON_Numerator'] = df['INTRON_Numerator'].astype(int)
df['INTRON_Denominator'] = df['INTRON_Denominator'].astype(int)

# Initialize an empty DataFrame to store the final result
result_df = pd.DataFrame()

# Iterate over unique combinations of CHROM, POS, rsID, REF, ALT
for _, group_df in df.groupby(['CHROM', 'POS', 'REF', 'ALT']):
    # Check if EXON column has values
    if not group_df['EXON_Numerator'].eq(0).all():
        # Prioritize rows with values in EXON column
        result_df = pd.concat([result_df, group_df.sort_values(by=['EXON_Numerator'], ascending=False).head(1)])
    else:
        # If EXON is empty, prioritize rows with values in INTRON column
        if not group_df['INTRON_Numerator'].eq(0).all():
            result_df = pd.concat([result_df, group_df.sort_values(by=['INTRON_Numerator'], ascending=False).head(1)])
        else:
            # If both EXON and INTRON are empty, just concatenate the first row
            result_df = pd.concat([result_df, group_df.head(1)])

# Now result_df contains the rows you're looking for based on the specified logic
result_df = result_df.drop(['Gene_Match', 'Gene Name', 'EXON_Numerator', 'EXON_Denominator', 'INTRON_Numerator', 'INTRON_Denominator'], axis=1)

result_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL39.xlsx', index=False)

In [1]:
import os
import pandas as pd

# Set the path to the folder containing your Excel files
folder_path = r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/latest_outputs/'

# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the Excel file into a DataFrame
        df = pd.read_excel(file_path)
        
        # Extract the "Sample" value from the base name of the file
        sample_name = os.path.splitext(filename)[0]
        
        # Add a new column "Sample" with the extracted value
        df['Sample'] = sample_name
                
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list
concatenated_data = pd.concat(dfs, ignore_index=True)

# Save the final DataFrame to a new Excel file
output_file_path = r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/latest_outputs/All_samples_concatenated_data.xlsx'
concatenated_data.to_excel(output_file_path, index=False)

# Print the concatenated DataFrame
concatenated_data

,Matched_Gene,rsID,Literature,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,...,MAX_AF,MAX_AF_POPS,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,Sample
0,PCSK9,rs45448095,No,chr1,55039774,C,T,Heterozygous,5 prime UTR variant,3/10,...,0.2934,gnomADg AMI,protein coding,1/12,0/0,NaN,NaN,1,"25741868,19191301,12730697,29748315,33889589,3...",KHCDPRGPTTL1
1,PCSK9,rs11583680,No,chr1,55039995,C,T,Heterozygous,missense variant,7/10,...,0.2941,gnomADg AMI,protein coding,1/12,0/0,A53V,gCc/gTc,1,"25741868,28966647,25266949,30473376,31036026,2...",KHCDPRGPTTL1
2,PCSK9,rs2495482,No,chr1,55040059,A,G,Homozygous,intron variant,2/10,...,0.9816,SAS,protein coding,0/0,1/11,NaN,NaN,1,"25741868,17142622",KHCDPRGPTTL1
3,PCSK9,rs625619,No,chr1,55052493,G,A,Homozygous,"intron variant,NMD transcript variant",2/10,...,0.8046,EAS,nonsense mediated decay,0/0,5/5,NaN,NaN,1,"25741868,18300938",KHCDPRGPTTL1
4,PCSK9,rs509504,No,chr1,55057360,A,G,Homozygous,synonymous variant,3/10,...,1.0000,"EAS,EUR,SAS,gnomADe ASJ,gnomADe EAS,gnomADe FI...",protein coding,7/12,0/0,Q342,caA/caG,1,25741868,KHCDPRGPTTL1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25343,ABCA1,rs2230808,No,chr9,104800523,T,C,Heterozygous,missense variant,7/10,...,0.8451,gnomADg FIN,protein coding,35/50,0/0,K1587R,aAa/aGa,-1,"21467728,22833659,23656756,19041386,19878569,3...",KHCDPRGPTTL9
25344,ABCA1,rs2066716,No,chr9,104806424,C,T,Heterozygous,synonymous variant,3/10,...,0.2938,gnomADg EAS,protein coding,31/50,0/0,T1427,acG/acA,-1,"23018797,23168575,20167577,17855807,35213538",KHCDPRGPTTL9
25345,ABCA1,rs2246841,No,chr9,104840385,C,T,Heterozygous,synonymous variant,3/10,...,0.2685,AFR,protein coding,9/50,0/0,G316,ggG/ggA,-1,17553166,KHCDPRGPTTL9
25346,ABCA1,rs2230806,Yes,chr9,104858586,C,T,Heterozygous,missense variant,7/10,...,0.7103,AFR,protein coding,7/50,0/0,R219K,aGg/aAg,-1,"23021345,23497168,28690685,20185793,30584432,2...",KHCDPRGPTTL9


In [2]:
concatenated_data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/latest_outputs/All_samples_concatenated_data.xlsx')
concatenated_data

,Matched_Gene,rsID,Literature,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,...,MAX_AF,MAX_AF_POPS,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,Sample
0,PCSK9,rs45448095,No,chr1,55039774,C,T,Heterozygous,5 prime UTR variant,3/10,...,0.2934,gnomADg AMI,protein coding,1/12,0/0,NaN,NaN,1,"25741868,19191301,12730697,29748315,33889589,3...",KHCDPRGPTTL1
1,PCSK9,rs11583680,No,chr1,55039995,C,T,Heterozygous,missense variant,7/10,...,0.2941,gnomADg AMI,protein coding,1/12,0/0,A53V,gCc/gTc,1,"25741868,28966647,25266949,30473376,31036026,2...",KHCDPRGPTTL1
2,PCSK9,rs2495482,No,chr1,55040059,A,G,Homozygous,intron variant,2/10,...,0.9816,SAS,protein coding,0/0,1/11,NaN,NaN,1,"25741868,17142622",KHCDPRGPTTL1
3,PCSK9,rs625619,No,chr1,55052493,G,A,Homozygous,"intron variant,NMD transcript variant",2/10,...,0.8046,EAS,nonsense mediated decay,0/0,5/5,NaN,NaN,1,"25741868,18300938",KHCDPRGPTTL1
4,PCSK9,rs509504,No,chr1,55057360,A,G,Homozygous,synonymous variant,3/10,...,1.0000,"EAS,EUR,SAS,gnomADe ASJ,gnomADe EAS,gnomADe FI...",protein coding,7/12,0/0,Q342,caA/caG,1,25741868,KHCDPRGPTTL1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25343,ABCA1,rs2230808,No,chr9,104800523,T,C,Heterozygous,missense variant,7/10,...,0.8451,gnomADg FIN,protein coding,35/50,0/0,K1587R,aAa/aGa,-1,"21467728,22833659,23656756,19041386,19878569,3...",KHCDPRGPTTL9
25344,ABCA1,rs2066716,No,chr9,104806424,C,T,Heterozygous,synonymous variant,3/10,...,0.2938,gnomADg EAS,protein coding,31/50,0/0,T1427,acG/acA,-1,"23018797,23168575,20167577,17855807,35213538",KHCDPRGPTTL9
25345,ABCA1,rs2246841,No,chr9,104840385,C,T,Heterozygous,synonymous variant,3/10,...,0.2685,AFR,protein coding,9/50,0/0,G316,ggG/ggA,-1,17553166,KHCDPRGPTTL9
25346,ABCA1,rs2230806,Yes,chr9,104858586,C,T,Heterozygous,missense variant,7/10,...,0.7103,AFR,protein coding,7/50,0/0,R219K,aGg/aAg,-1,"23021345,23497168,28690685,20185793,30584432,2...",KHCDPRGPTTL9


In [3]:
gene_summary = concatenated_data.groupby(['Matched_Gene', 'rsID']).agg({'Sample': lambda x: ', '.join(x.unique())}).reset_index()
gene_summary['Sample_count'] = gene_summary['Sample'].apply(lambda x: len(x.split(',')))
gene_summary = gene_summary[['Matched_Gene', 'rsID', 'Sample', 'Sample_count']]
gene_summary

,Matched_Gene,rsID,Sample,Sample_count
0,ABCA1,rs138880920,"KHCDPRGPTTL34, KHCDPRGPTTL47, KHCDPRGPTTL60, K...",4
1,ABCA1,rs142877738,"KHCDPRGPTTL67, KHCDPRGPTTL81",2
2,ABCA1,rs150520154,KHCDPRGPTTL67,1
3,ABCA1,rs1800978,"KHCDPRGPTTL10, KHCDPRGPTTL11, KHCDPRGPTTL12, K...",42
4,ABCA1,rs182899836,KHCDPRGPTTL65,1
...,...,...,...,...
1832,VCL,rs548487697,"KHCDPRGPTTL17, KHCDPRGPTTL27",2
1833,VCL,rs551463686,KHCDPRGPTTL36,1
1834,VCL,rs703258,"KHCDPRGPTTL1, KHCDPRGPTTL10, KHCDPRGPTTL11, KH...",58
1835,VCL,rs767809,"KHCDPRGPTTL1, KHCDPRGPTTL10, KHCDPRGPTTL11, KH...",51


In [4]:
gene_summary.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/latest_outputs/Gene_rsID_sample_counts.xlsx', index = False)

In [17]:
import pandas as pd

# Assuming concatenated_data is your DataFrame
df = concatenated_data.copy()

# Create a new column by concatenating "Matched_Gene" and "rsID"
df['Combined'] = df['Matched_Gene'].astype(str) + '_' + df['rsID'].astype(str)

# Get the set of unique combinations for each sample
sample_combinations = df.groupby('Sample')['Combined'].unique()

# Find the common combinations across all samples
common_combinations = set(sample_combinations.iloc[0]).intersection(*sample_combinations[1:])

# Filter the original DataFrame to include only the common combinations
result_df = df[df['Combined'].isin(common_combinations)]

# Print or save the result
result_df

,Matched_Gene,rsID,Literature,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,Sample,Combined
31,RYR2,rs707189,No,chr1,237638470,T,C,Homozygous,synonymous variant,3/10,LOW,2.5,"Cardiovascular phenotype,Cardiac arrhythmia,Ca...",benign,"criteria provided, multiple submitters, no con...",43819.0,ENST00000366574.7,c.6906T>C,ENSP00000355533.2,p.Leu2302%3D,1/1,164,29,29,0,29,100%,3.325900e-17,0,0,21,8,NaN,NaN,0.9549,0.8359,0.9870,1.0000,1.0000,1.0000,0.9906,0.8674,0.9936,1.0000,1.0000,0.9999,0.9998,0.9965,0.9997,0.9642,0.8757,1.0000,0.9855,1.0000,1.0000,1.0000,1.0000,0.9997,0.9761,1.0000,1.0000,"EAS,EUR,SAS,gnomADe ASJ,gnomADe EAS,gnomADg AM...",protein coding,45/105,0/0,L2302,ctT/ctC,1,"25741868,24033266",KHCDPRGPTTL1,RYR2_rs707189
33,RYR2,rs2797436,No,chr1,237700418,T,G,Homozygous,synonymous variant,3/10,LOW,2.5,"Cardiovascular phenotype,Cardiac arrhythmia,Ca...",benign,"criteria provided, multiple submitters, no con...",43836.0,ENST00000366574.7,c.9318T>G,ENSP00000355533.2,p.Ser3106%3D,1/1,158,28,28,0,28,100%,1.307400e-16,0,0,24,4,NaN,NaN,0.9692,0.8880,0.9914,1.0000,1.0000,1.0000,0.9928,0.9012,0.9953,1.0000,0.9997,1.0000,0.9994,0.9951,0.9993,0.9717,0.9021,0.9989,0.9887,1.0000,1.0000,1.0000,0.9968,0.9995,0.9813,0.9990,1.0000,"EAS,EUR,SAS,gnomADe ASJ,gnomADe FIN,gnomADg AS...",protein coding,65/105,0/0,S3106,tcT/tcG,1,"25741868,24033266,30019117",KHCDPRGPTTL1,RYR2_rs2797436
51,RBM20,rs6585012,No,chr10,110783459,G,A,Homozygous,intron variant,2/10,MODIFIER,1.5,Dilated cardiomyopathy 1DD,benign,"criteria provided, multiple submitters, no con...",671182.0,ENST00000369519.4,c.1337+32G>A,NaN,None,1/1,76,14,14,0,14,100%,2.492700e-08,0,0,10,4,NaN,NaN,0.7881,0.5840,0.8256,0.9673,0.7575,0.8845,0.8181,0.6227,0.8327,0.7746,0.9687,0.8326,0.7968,0.7885,0.8677,0.7589,0.6219,0.8121,0.7978,0.7679,0.9717,0.8285,0.7310,0.7988,0.7304,0.8628,0.9717,gnomADg EAS,protein coding,0/0,3/13,NaN,NaN,1,NaN,KHCDPRGPTTL1,RBM20_rs6585012
60,MYBPC3,rs896818,No,chr11,47343211,A,G,Homozygous,"intron variant,NMD transcript variant",2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1232915.0,ENST00000544791.1,c.1226+49T>C,NaN,None,1/1,87,16,16,0,16,100%,1.663700e-09,0,0,11,5,NaN,NaN,0.9641,0.8729,0.9827,1.0000,1.0000,1.0000,0.9911,0.8885,0.9925,0.9958,0.9999,0.9999,0.9992,0.9940,0.9994,0.9692,0.8952,1.0000,0.9851,0.9965,1.0000,1.0000,0.9589,0.9993,0.9765,0.9998,1.0000,"EAS,EUR,SAS,gnomADg AMI,gnomADg EAS,gnomADg FIN",nonsense mediated decay,0/0,14/26,NaN,NaN,-1,18414213,KHCDPRGPTTL1,MYBPC3_rs896818
67,SCN3B,rs1148110,No,chr11,123634251,C,G,Homozygous,"intron variant,NMD transcript variant",2/10,MODIFIER,1.5,Brugada syndrome 7,benign,"criteria provided, multiple submitters, no con...",670950.0,ENST00000667790.1,c.*566-45G>C,NaN,None,1/1,87,16,16,0,16,100%,1.663700e-09,0,0,10,6,NaN,NaN,0.7506,0.9720,0.6225,0.7222,0.5467,0.7812,0.6196,0.9125,0.5962,0.6414,0.7529,0.4855,0.5526,0.5865,0.7499,0.6672,0.9045,0.5923,0.6404,0.6413,0.7482,0.4801,0.6646,0.5469,0.6579,0.7698,0.9720,AFR,nonsense mediated decay,0/0,6/7,NaN,NaN,-1,NaN,KHCDPRGPTTL1,SCN3B_rs1148110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11566,PKP2,rs6

In [16]:
result_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/Common_genes_rsID_combinations.xlsx', index=False)

In [19]:
import os
import pandas as pd

# Set the path to the folder containing your Excel files
folder_path = r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/'

# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)

        # Read the Excel file into a DataFrame
        df = pd.read_excel(file_path)

        # Extract the "Matched_Gene" and "rsID" columns
        subset_df = df[['Matched_Gene', 'rsID']]

        # Add a new column "Sample" with the extracted value
        subset_df['Sample'] = os.path.splitext(filename)[0]

        # Append the subset DataFrame to the list
        dfs.append(subset_df)

# Concatenate all subset DataFrames in the list
concatenated_subset_data = pd.concat(dfs, ignore_index=True)

# Find common values in "Matched_Gene" and "rsID" columns across all samples
common_genes = set(concatenated_subset_data['Matched_Gene'])
common_rsids = set(concatenated_subset_data['rsID'])

# Convert sets to DataFrames
common_genes_df = pd.DataFrame(list(common_genes), columns=['Common_Genes'])
common_rsids_df = pd.DataFrame(list(common_rsids), columns=['Common_rsIDs'])

# Save DataFrames to Excel files
common_genes_file_path = r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/common_genes.xlsx'
common_rsids_file_path = r'C:/Users/GenepoweRx_Madhu/Downloads/KHCDPRGPTTL_VCFs/OUTPUT_data/new_output/common_rsids.xlsx'

common_genes_df.to_excel(common_genes_file_path, index=False)
common_rsids_df.to_excel(common_rsids_file_path, index=False)


In [2]:
import pandas as pd

# Sample DataFrame
data = {'CHROM': ['chr2', 'chr1', 'chr1', 'chr3', 'chr2'],
        'POS': [500, 200, 300, 100, 400],
        'Other_Column': ['A', 'B', 'C', 'D', 'E']}
df = pd.DataFrame(data)
# Sort based on 'CHROM' and 'POS'
df.sort_values(by=['CHROM', 'POS'], inplace=True)
df

,CHROM,POS,Other_Column
1,chr1,200,B
2,chr1,300,C
4,chr2,400,E
0,chr2,500,A
3,chr3,100,D


In [1]:
import pandas as pd

# Example data
data = {
    'rsID': ['rs1', 'rs2', 'rs3', 'rs4', 'rs5'],
    'distance_1': [10, 5, 8, 15, 20],
    'distance_2': [12, 6, 9, 14, 18],
    'distance_3': [11, 7, 10, 13, 19],
    'distance_4': [13, 8, 11, 12, 17],
    'distance_5': [9, 4, 7, 16, 21]
}

df = pd.DataFrame(data)

# Extracting distance column names
distance_columns = df.columns[1:]

# Create a new column for the nearest distance column name
df['nearest_distance_column'] = df.apply(lambda row: min(distance_columns, key=lambda col: row[col]), axis=1)

# Display the resulting DataFrame
df

,rsID,distance_1,distance_2,distance_3,distance_4,distance_5,nearest_distance_column
0,rs1,10,12,11,13,9,distance_5
1,rs2,5,6,7,8,4,distance_5
2,rs3,8,9,10,11,7,distance_5
3,rs4,15,14,13,12,16,distance_4
4,rs5,20,18,19,17,21,distance_4
